In [27]:
import pywt

import torch
import torch.nn  as nn

import meshplot 

from src.shared.faces import make_cube_faces
from src.shared.sides import (    
    to_vertices,
    to_stacked,
    make_phi_theta,    
)
torch.set_printoptions(precision=5, sci_mode=False)

In [20]:
v = torch.load('./coarse_128.pt')
vert = to_stacked(v)
n = 128
f = make_cube_faces(n)
v.shape, f.shape

(torch.Size([98304, 3]), torch.Size([196604, 3]))

In [15]:
meshplot.plot(vert.numpy(), f.numpy());

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.053305…

In [79]:
import torch
from pytorch_wavelets import DWTForward, DWTInverse # (or import DWT, IDWT)
xfm = DWTForward(J=3, mode='zero', wave='db5')  # Accepts all wave types available to PyWavelets
ifm = DWTInverse(mode='zero', wave='db5')

Yl, Yh = xfm(vert)
print(Yl.shape)
print(Yh[0].shape)
print(Yh[1].shape)
print(Yh[2].shape)
rec_vert = ifm((Yl, Yh))

torch.testing.assert_allclose(vert,  rec_vert)
#meshplot.plot(to_vertices(rec_vert).numpy(), f.numpy());

torch.Size([6, 3, 23, 23])
torch.Size([6, 3, 3, 68, 68])
torch.Size([6, 3, 3, 38, 38])
torch.Size([6, 3, 3, 23, 23])


In [80]:
values, _ = Yh[0].flatten().abs().sort(descending=True)
values

tensor([0.33113, 0.33113, 0.33043,  ..., 0.00000, 0.00000, 0.00000])

In [81]:
threshold = values[int(len(values) * 0.5)]
threshold

tensor(0.00025)

In [82]:
from  math import floor

def compress(t, keep=1):
    values, _ = t.flatten().abs().sort(descending=True)
    threshold_index = int(floor((1-keep)*len(values))) 
    print(threshold_index, len(values))
    threshold = values[threshold_index]
    print(values.abs().max(), threshold)
    #threshold = values[int(floor((1-keep)*len(values)))]
    indices = t.abs() <= threshold
    filtered = t * indices # Threshold small indices
    return filtered#, indices

#print(Yh[0].shape)
#print(Yh[1].shape)
#print(Yh[2].shape)
filtered = [compress(t, 0.25) for t in Yh]
rec_vert = ifm((Yl, filtered))

meshplot.plot(to_vertices(rec_vert).numpy(), f.numpy());
#torch.testing.assert_allclose(vert,  rec_vert)

187272 249696
tensor(0.33113) tensor(    0.00002)
58482 77976
tensor(0.37494) tensor(0.00012)
21424 28566
tensor(1.10504) tensor(0.00039)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.018744…

In [56]:
keep = 1
t = Yh[0]

values, _ = t.flatten().abs().sort(descending=False)
    
threshold = values[int(np.floor((1-keep)*len(values)))]
print(values.abs().max(), threshold)
#threshold = values[int(floor((1-keep)*len(values)))]
indices = t.abs() <= threshold
filtered = t * indices # Threshold small indices
indices

tensor(0.37569) tensor(0.)


tensor([[[[[False, False, False,  ..., False, False, False],
           [False, False, False,  ..., False, False, False],
           [False, False, False,  ..., False, False, False],
           ...,
           [False, False, False,  ..., False, False, False],
           [False, False, False,  ..., False, False, False],
           [False, False, False,  ..., False, False, False]],

          [[False, False, False,  ..., False, False, False],
           [False, False, False,  ..., False, False, False],
           [False, False, False,  ..., False, False, False],
           ...,
           [False, False, False,  ..., False, False, False],
           [False, False, False,  ..., False, False, False],
           [False, False, False,  ..., False, False, False]],

          [[False, False, False,  ..., False, False, False],
           [False, False, False,  ..., False, False, False],
           [False, False, False,  ..., False, False, False],
           ...,
           [False, False, False, 

NameError: name 'torch' is not defined